# Exposure Time Calculator tutorial


In [ ]:
# Allows interactive plot within this notebook
%matplotlib notebook

# Allows to take into account modifications made in the source code without having to restart the notebook
%reload_ext autoreload
%autoreload 2

# Telescope configuration Files

First you need to define the caracteristics of your telescope in a hjson file. This file must be in pyETC/pyETC/telescope_database/.

This file contains all caracteristics of the telescope and its environment you do not need to modify for different observations. (Size of the mirrors, number ad type of lenses, mirrors..., sky background, atmosphere transmission, cameras caracteristics,...)

For instance for the COLIBRI: [COLIBRI.hjson](../pyETC/telescope_database/COLIBRI.hjson)

# ETC configuration File 

The ETC configuration file contains information related to the conditions of an observation: moon age, target elevation, seeing, filter band to use, exposure time, number of exposures...

An example of configuration file can be found in in pyETC/pyETC/configFiles: [example.hjson](../pyETC/configFiles/example.hjson)



# Simple user cases

## 1) Illustrative example

In order to compute the limiting magnitudes you need to define a SNR and an exposure time

In [ ]:
# Load os package, in order to get the environment variable GFT_SIM
import os

# Load ETC package
from pyETC.pyETC import etc

In [ ]:
# load ETC with a config file and the COLIBRI caracteristics
COLIBRI_ETC=etc(configFile='example.hjson',name_telescope='colibri')

In [ ]:
# Execute
COLIBRI_ETC.sim()

The plots and results displayed here are also saved in a .txt file in pyETC/results/results_summary.txt

The main idea of the ETC is that all information are stored in a dictionnary named "information".
If you want to know the whole content just write:

In [ ]:
COLIBRI_ETC.information

If you want to get the value of one parameter, for instance the computed magnitude, just write:

In [ ]:
COLIBRI_ETC.information['mag']

If you want to modify some parameters, there are 2 possibilities:   
- either modify the hjson configFile     
- or directly change the parameters value in the "information" dictionary. This is useful when the ETC is used in a python script interacting with other python packages.

For instance, we want to change the exposure time, SNR, seeing, elevation of the target, age of the moon without modifying the input file: 

In [ ]:
# Modify the exposure time
COLIBRI_ETC.information["exptime"] = 60

# Change the number of expositions
COLIBRI_ETC.information['Nexp'] = 3

# Change the seeing at the zenith, in arcseconds
COLIBRI_ETC.information["seeing_zenith"] =1.2

# Change the elevation of the target, in degrees 
COLIBRI_ETC.information['elevation'] = 78

#Change the age of the moon
COLIBRI_ETC.information["moon_age"] = 2

In [ ]:
# Execute again
COLIBRI_ETC.sim()

In the case of COLIBRI, there 3 channels and one might want to use one NIR band. You just need to specify the channel to use and the filter band, for instance the J band:

In [ ]:
# Select the J filter band
COLIBRI_ETC.information["filter_band"] = 'J'

# Specify the NIR channel in order to load the NIR channel caracterisitcs (optics transmissions + camera carac.)
COLIBRI_ETC.information["channel"] = 'CAGIRE'

In [ ]:
# Execute again
COLIBRI_ETC.sim()

## 2) Compute the SNR for a given magnitude (or spectrum) and a exposure time

### 2.1) For a given magnitude

To compute the SNR for a Vega magnitude of 18 and a single exposure of 5s, we need to modify in the hjson configFile:   
"etc_type": snr   
"object_type": magnitude   
"object_magnitude": 18   
"exptime": 5   
"photometry_system": Vega   

 
Here we rather update the dictionary to avoid to use too many configFiles.   


In [ ]:
# Now we want to compute the SNR
COLIBRI_ETC.information['etc_type'] = 'snr'

# Set up the object: either 'magnitude' for a constant magnitude,
# or 'spectrum' for a given spectrum in the database,
# or 'grb_sim' to ompute the grb spectrum
COLIBRI_ETC.information['object_type'] = 'magnitude'

# If we select 'magnitude', we need to define the object magnitude 
COLIBRI_ETC.information['object_magnitude'] = 18

# Set an exposure time of 5s
COLIBRI_ETC.information['exptime'] = 5

# Use Vega system
COLIBRI_ETC.information['photometry_system'] = 'Vega'



In [ ]:
# Execute
COLIBRI_ETC.sim()

### 2.2) For a given spectrum

The spectrum is stored in /data/   
Wavelength are in Angstoms and fluxes in erg/s/cm2/A 

In [ ]:
# Specify that the object is a spectrum
COLIBRI_ETC.information['object_type'] = 'spectrum'

# Define the folder. Starting from pyETC/pyETC directory
COLIBRI_ETC.information['object_folder'] = '/data/calspec/'

# Define the file in this folder
COLIBRI_ETC.information['object_file'] = 'bd02d3375_stis_001.txt'

#COLIBRI_ETC.information['object_folder'] = '/data/'
#COLIBRI_ETC.information['object_file']='bohlin2006_Vega.dat'


# Modify the exposure time
COLIBRI_ETC.information["exptime"] = 5

# Change the number of expositions
COLIBRI_ETC.information['Nexp'] = 3

# Change the seeing at the zenith, in arcseconds
COLIBRI_ETC.information["seeing_zenith"] = 0.79

# Change the elevation of the target 
COLIBRI_ETC.information['elevation'] = 78

#Change the age of the moon
COLIBRI_ETC.information["moon_age"] = 7

# Specify the channel
COLIBRI_ETC.information["channel"]= 'DDRAGO-R'

# Select the z filter band
COLIBRI_ETC.information["filter_band"] = 'z'

# Use AB system
COLIBRI_ETC.information['photometry_system'] = 'AB'

In [ ]:
# Execute
COLIBRI_ETC.sim()

### 2.3) For a simulated GRB spectrum 

(Required pyGRBaglow python package)

GRB spectrum can be simulated with:   
- empirical model: single power law, broken power law
- theoretical model: synchrotron model of Granot & Sari 2002

In the following we use the theoretical model.  
In the following we update the dictionary, but one can also load the tuto_grb_sim.hjson configFile.

In [ ]:
# First we specify that the object is a simulated GRB spectrum
COLIBRI_ETC.information['object_type'] = 'grb_sim'

# Specify the GRB model to use
COLIBRI_ETC.information['grb_model'] = 'gs02'

# Redshift
COLIBRI_ETC.information['grb_redshift'] = 3

# Time (in days)
COLIBRI_ETC.information['t_sinceBurst'] = 0.2

# Equivalent isotropic energy
COLIBRI_ETC.information['E_iso'] = 1e53

# Gamma-ray radiative efficiency
COLIBRI_ETC.information['eta'] = 0.3

# fraction of the internal energy given to the magnetic field in the Forward Shock
COLIBRI_ETC.information['eps_b'] = 1e-4

# fraction of the internal energy given to the electrons accelerated into the Forward Shock
COLIBRI_ETC.information['eps_e'] = 0.1

# index of the energy distribution of the shocked accelerated electrons
COLIBRI_ETC.information['p'] = 2.2

# interstellar medium density (in cm3)
COLIBRI_ETC.information['n0'] = 1

# Inverse Compton parameter, to take into account the Inverse Compton effects on the cooling of electrons
COLIBRI_ETC.information['Y'] = 0

# ISM type: 0: constant ISM density / 1: Massive star progenitor surounded by its preexplosion Wind
COLIBRI_ETC.information['ism_type'] = 0


# Host galaxy extinction (either 'mw', 'smc','lmc' or 'none')
COLIBRI_ETC.information['host_extinction_law'] = 'smc'
# Amount of extinction in the V band (in mag)
COLIBRI_ETC.information['Av_Host'] = 0.2 
COLIBRI_ETC.information['galactic_extinction_law'] = 'smc'

# IGM extinction model: either 'madau' or 'meiksin' or 'none'
COLIBRI_ETC.information['IGM_extinction_model'] = 'meiksin'

# Galactic extinction, by default a mw extinction law is used
# Amount of galactic extinction in V band (in mag)
COLIBRI_ETC.information['Av_galactic'] = 0.1



In [ ]:
#Execute
COLIBRI_ETC.sim()

## 3) Compute exposure time for a given SNR and magnitude or spectrum

Here we compute the exposure time to reach a magnitude of 18 (AB system) in z band with a SNR of 10 with the COLIBRI telescope.

In [ ]:
# Specify that we want to compute the exposure time
COLIBRI_ETC.information['etc_type'] = 'time'

# For a given magnitude
COLIBRI_ETC.information['object_type'] = 'magnitude'

# Define the object magnitude 
COLIBRI_ETC.information['object_magnitude'] = 18

# Define the SNR
COLIBRI_ETC.information['SNR'] = 10

# Specify the channel
COLIBRI_ETC.information["channel"]= 'DDRAGO-R'

# Select the z filter band
COLIBRI_ETC.information["filter_band"] = 'z'

# Use AB system
COLIBRI_ETC.information['photometry_system'] = 'AB'

#If you do not want to display the plot:
COLIBRI_ETC.information['plot'] = False

In [ ]:
# Execute
COLIBRI_ETC.sim()

## 4) Compute limiting magnitudes

Here we want to compute the limiting magnitude for a SNR = 10 and 3 exposures of 10s in r band with the COLIBRI telescope.

In [ ]:
# Specify that we want to compute the exposure time
COLIBRI_ETC.information['etc_type'] = 'mag'

# Define the SNR
COLIBRI_ETC.information['SNR'] = 10

# Modify the exposure time
COLIBRI_ETC.information["exptime"] = 10

# Change the number of expositions
COLIBRI_ETC.information['Nexp'] = 3

# Change the seeing at the zenith, in arcseconds
COLIBRI_ETC.information["seeing_zenith"] = 0.79

#Change the age of the moon
COLIBRI_ETC.information["moon_age"] = 7

# Specify the channel
COLIBRI_ETC.information["channel"]= 'DDRAGO-B'

# Select the z filter band
COLIBRI_ETC.information["filter_band"] = 'r'

# Use AB system
COLIBRI_ETC.information['photometry_system'] = 'AB'

#If you do not want to display the plot:
COLIBRI_ETC.information['plot'] = False

In [ ]:
COLIBRI_ETC.sim()

## 5) Other

In [ ]:
# If you do not want to display verbose:
COLIBRI_ETC.information['verbose'] = False

# If you do not want to create plots:
COLIBRI_ETC.information['plot'] = False


In [ ]:
# Write system transmission of the last run in a file named 'sys_trans.txt' with wavelength in nm
COLIBRI_ETC.write_file_trans(COLIBRI_ETC.information['system_response'],'sys_trans',wvl_unit='nm')

In [ ]:
# Plot system transmission  of last run
trans=COLIBRI_ETC.information['system_response']
COLIBRI_ETC.plot_trans(trans,'system_transmision',title='test system response',ylabel='Transmission',
                   ylim=[0,1],wvl_unit='microns',passband_centered=True)